## Import Azure ML SDK

In [ ]:
import azureml.core
from azureml.core import Workspace
from azureml.core.model import Model
from azureml.core import Experiment
from azureml.core.webservice import Webservice
from azureml.core.image import ContainerImage
from azureml.core.webservice import AciWebservice
from azureml.core.conda_dependencies import CondaDependencies

## Check  Azure SDK version

In [ ]:
print(azureml.core.VERSION)

## Create Azure ML Workspace

In [ ]:
ws = Workspace.create(name='salary', 
subscription_id='c15797£7-334d-4759-b8fe-a0c7b908e"
resource_group='webinar', 
create_resource_group=True, 
location='southeastasia'
) 

## Write Configuration to local file

In [ ]:
ws.write_config()

## Access from existing config file

In [ ]:
ws = Workspace.from_config()

In [ ]:
ws.get_details()

## Create Azure ML Experiment

In [ ]:
exp = Experiment(workspace=ws, name='salexp')

## Start logging metrics

In [ ]:
run = exp.start_logging()
run.log("Experiment start time", str(datetime.datetime.now()))

## Train your ML model (Code)

## Freeze the model

In [ ]:
filename = 'outputs/sal_model.pkl'
joblib.dump(lm, filename)

## Test Model

In [ ]:
filename = 'outputs/sal_model.pkl'
loaded_model=joblib.load(filename) 
y=loaded_model.predict([[21]])[0] 
print(y)

## Log metrics to Azure ML Experiment

In [ ]:
run.log('Intercept :', lm.intercept )
run.log('Slope :', lm.coef_[0])

## End Azure ML Experiment

In [ ]:
run.log("Experiment end time", str(datetime.datetime.now()))
run.complete()

## Get Portal URL

In [ ]:
print(run.get_portal_url())

## Register the model

In [ ]:
model = Model.register(model_path = "outputs/sal_model.pkl",
                       model_name = "sal_model”,
                       tags = {"key": "1"},
                       description = "Salary Prediction”,
                       workspace = ws)

## Define Azure ML Deploymemt configuration

In [ ]:
aciconfig = AciWebservice.deploy_configuration(cpu_cores=1,
                                               memory_gb=1,
                                               tags={"data": "salary","": "method":"sklearn"}
                                               description='Predict Stack

## Create enviroment configuration file

In [ ]:
salenv = CondaDependencies()
salenv.add_conda_package("scikit-learn")

with open("salenv.yml","w") as f:
    f.write(salenv.serialize_to_string())
with open("salenv.yml","r") as f:
    print(f.read())

## Create Azure ML Scoring file

In [ ]:
%%writefile score.py

import json
import numpy as np
import os
import pickle 
from sklearn.externals import joblib
from sklearn.linear_model import LogisticRegression
from azureml.core.model import Model


def init():
    global model
    # retrive the path to the modelfile using model name
    model_path = Model.get_model_path("sal_model")
    model =joblib.load(model_path)
    
def run(raw_data):
    data = np.array(json.loads(raw_data)['data'])
    #make prediction
    y_hat = model.predict(data)
    return json.dumps(y_hat.tolist())

## Deploy the model to Azure Container Instance

In [ ]:
image_config = ContainerImage.image_configuration(execution_script="score.py",
                                                  runtime="python",
                                                  conda_file="salenv.yml")

## Expose web service

In [ ]:
service = Webservice.deploy_from model(workspace=ws,
                                       name='salary-svc',
                                       deployment_config=aciconfig,
                                       models=[model],
                                       image_config=image_config)
service.wait_for_deployment(show_output=True)

In [ ]:
service.wait_for_deployment(show_output=True)

## Get the Web Service URL

In [ ]:
print(service.scoring_uri)

In [ ]:
!curl -X POST \
    -H 'Content-Type':'application/json' \
    -d '{"data":[[45]]}' \
        http://9fdcdd13-2eda-4ad6-812e-d2£330£a46c3. southeastasia.azurecontai

## Delete Workspace and clean up resources

In [ ]:
ws.delete()